# LDA 주제분석

In [1]:
import sys
import gc
import os
import re
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
# import chromedriver_autoinstaller

from matplotlib import font_manager, rc
# 차트에서 한글 출력을 위한 설정

import matplotlib.pyplot as plt
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common. by import By

from bs4 import BeautifulSoup 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import konlpy
from konlpy.tag import Okt
from konlp.kma.klt2023 import klt2023
import nltk
import stanza
from gensim.models import Word2Vec, word2vec
from gensim.models import FastText
from gensim.matutils import Sparse2Corpus
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel

from wordcloud import WordCloud
from numpy import dot
from numpy.linalg import norm
import itertools

## 전처리

In [3]:
# 중복 제거 및 데이터 불러오기
df = pd.read_csv('book_all.csv').drop_duplicates().reset_index(drop=True)

In [4]:
# 책소개에 이모티콘과 문장 부호가 많아, 더 깔끔한 데이터 확보를 위해 제거해줌.

for i in range(len(df)):
    df['책소개'][i] = re.sub("[^A-Za-z0-9가-힣]",   # 바꿀패턴:영어, 숫자, 한글이 아닌 모든것 제거
                      " ",                   # 바뀐내용:공백으로 바꿔라
                      df['책소개'][i] )

In [5]:
content = df['책소개'].to_list()

In [6]:
ko_Stopword = pd.read_csv('한국어 불용어.csv')['불용어'].to_list()

Corpus 형식으로 변환

In [7]:
token_re = re.compile(r'\b\w{2,}\b', re.UNICODE)         # 두 글자 이상인 단어를 뽑아라
def tokenizer(text):
    text = text.lower()
    words = []
    for word in token_re.findall(text):
        if word not in ko_Stopword:
            words.append(word)
    return words

docs = []
for text in df.책소개:
    doc = tokenizer(text)
    docs.append(doc)

dic = Dictionary(docs)             # 단어 사전 생성

corpus = []
for doc in docs:
    bow = dic.doc2bow(doc)
    corpus.append(bow)

dic.filter_extremes(no_below=10, no_above=0.7)   # 최소 10개 문서에서 등장하고, 0.7 이상의 문서에서 등장하지 않는 단어를 filtering

corpus = []
for doc in docs:
    bow = dic.doc2bow(doc)    # 단어들이 몇개 나왔는지 계산 Countvectorizer와 비슷
    corpus.append(bow)

## 모델 학습

In [8]:
# 데이터 분할 (성능 향상을 위하여)
train_corpus, valid_corpus = train_test_split(corpus, test_size=0.1, random_state=0)

In [9]:
model = LdaModel(
    corpus=train_corpus,
    id2word=dic,
    num_topics=100,
    random_state=0)

모델 학습

In [10]:
old_loss = -np.inf
loss = model.log_perplexity(valid_corpus)
loss

-25.11174080964044

In [11]:
while loss > old_loss + 0.1:
    model.update(train_corpus)
    old_loss = loss
    loss = model.log_perplexity(valid_corpus)
    print(loss)

-20.766237251205524
-19.271851685439387
-18.729216791051265
-18.516725797824808
-18.429464923187307


In [12]:
model.save('lda-model')

## 시각화

In [13]:
p = pyLDAvis.gensim.prepare(model, corpus, dic, sort_topics=False)
pyLDAvis.display(p)

# 람다 =0.6
# 토픽에서 가장 많이 나온 단어 기준
# 평소에 나오는 비율 대비해서 이 토픽에서 얼마나 자주 나오느냐 기준